# Data Downloader

    Version: 1.0
    Date:    27/09/2019
    Author:  Ben Loveday (Plymouth Marine Laboratory) and Hayley Evers-King (EUMETSAT)
    Credit:  This code was developed for EUMETSAT under contracts for the Copernicus 
             programme.
    License: This code is offered as open source and free-to-use in the public domain, 
             with no warranty.

**What is this notebook for?**

This notebook will download EUMETSAT Sentinel-3 data to support plotting.

**What specific tools does this notebook use?**

The harmonised data access API

***

Python is divided into a series of modules that each contain a series of methods for specific tasks. The box below imports all of the moduls we need to complete our plotting task

In [1]:
# standard tools
import os, sys, json
from zipfile import ZipFile

# specific tools (which can be found here ../Hub_tools/)
sys.path.append(os.path.dirname(os.path.dirname(os.getcwd())) + '/Hub_Tools/')
import harmonised_data_access_api_tools as hapi

WEkEO provides access to a huge number of datasets through its 'harmonised-data-access' API. This allows us to query the full data catalogue and download data quickly and directly onto our Jupyter Hub. You can search for what data is available here: https://www.wekeo.eu/dataset-navigator/start.

In order to use the HDA-API we need to provide some authentication credentials, which comes in the form of an api_key. You can get your key from here; https://www.wekeo.eu/api-keys. If you click on the 'show hidden keys' button at the bottom of the page it will reveal a number of keys. The one you need is in the top grey box, and is on the following line:

-H "Authorization: Basic "**YOUR API KEY**"

Replace "YOUR API KEY" below with what you copy from "**YOUR API KEY**" (N.B. you need to keep the quotation marks.)

In [2]:
# your api key:
api_key = "cmJ1UGJQVzZnT09HU2RUWDJhTGFkOGY4RjhnYTpGRmFCTTNoSXluVk1NdEk4b2dPc2ZjMHFOdlVh"
# where the data should be downloaded to:
download_dir_path = "/home/jovyan/work/products/Dorian/"
# where we can find our data query form:
JSON_query_dir = os.path.join(os.getcwd(),'JSON_templates')
# HDA-API loud and noisy?
verbose = False

Make our download directory if it does not exist

In [3]:
if not os.path.exists(download_dir_path):
    os.makedirs(download_dir_path)

Set the data source (which we use as a key for our JSON query file)

In [4]:
# Sentinel-3 keys
dataset_ids = ["EO:EUM:DAT:SENTINEL-3:SL_1_RBT___",\
               "EO:EUM:DAT:SENTINEL-3:SL_1_RBT___",\
               "EO:EUM:DAT:SENTINEL-3:OL_1_EFR___",\
               "EO:EUM:DAT:SENTINEL-3:OL_2_WFR___",\
               "EO:EUM:DAT:SENTINEL-3:SR_2_WAT___",\
               "EO:EUM:DAT:SENTINEL-3:SR_2_WAT___",\
               "EO:EUM:DAT:SENTINEL-3:SR_2_WAT___",\
               "EO:EUM:DAT:SENTINEL-3:SR_2_WAT___"]

start_dates = ["2019-09-02T02:53:00.000Z","2019-09-03T03:06:00.000Z",\
               "2019-09-03T15:29:00.000Z","2019-09-02T15:16:00.000Z",\
               "2019-09-02T02:23:00.000Z","2019-09-02T14:59:00.000Z",\
               "2019-09-03T02:38:00.000Z","2019-09-03T15:12:00.000Z"]

end_dates = ["2019-09-02T02:57:00.000Z","2019-09-03T03:09:30.000Z",\
             "2019-09-03T15:33:00.000Z","2019-09-02T15:20:00.000Z",\
             "2019-09-02T03:13:00.000Z","2019-09-02T15:46:00.000Z",\
             "2019-09-03T03:26:00.000Z","2019-09-03T15:59:59.000Z"]

Now we download the data. See the following scripts and notebooks for information on how this works:
    
    *samples/How_To_Guide-Harmonized_Data_Access-v0.1.3.ipynb*
    *ocean-wekeo-jpyhub/HDA_API_Tools/HDA_API_downloading.ipynb*
    *ocean-wekeo-jpyhub/Hub_Tools/harmonised_data_access_api_tools.py*

In [5]:
for dataset_id, start_date, end_date in zip(dataset_ids, start_dates, end_dates):
    # find query file
    JSON_query_file = os.path.join(JSON_query_dir,"EO_EUM_DAT_SENTINEL-3_GENERAL.json")

    HAPI_dict = hapi.init(dataset_id, api_key, download_dir_path, verbose=verbose)
    HAPI_dict = hapi.get_access_token(HAPI_dict)
    HAPI_dict = hapi.accept_TandC(HAPI_dict)

    # load the query
    with open(JSON_query_file, 'r') as f:
        query = f.read()
        query = query.replace("%DATE_START%",start_date)
        query = query.replace("%DATE_END%",end_date)
        query = query.replace("%KEY%",dataset_id)
        query = json.loads(query)

    if verbose:
        print(query)
    
    # launch job
    HAPI_dict = hapi.launch_query(HAPI_dict, query)

    # wait for jobs to complete
    HAPI_dict = hapi.check_job_status(HAPI_dict)
    if HAPI_dict['nresults'] == 0:
        print('Nothing to do for this query....')
    else:
        HAPI_dict = hapi.get_results_list(HAPI_dict)
        HAPI_dict = hapi.get_download_links(HAPI_dict)
        HAPI_dict = hapi.download_data(HAPI_dict, skip_existing=True)
    
    # unzip file
    for filename in HAPI_dict['filenames']:
        if os.path.splitext(filename)[-1] == '.zip':
            print('Unzipping file')
            with ZipFile(filename, 'r') as zipObj:
                # Extract all the contents of zip file in current directory
                zipObj.extractall(os.path.dirname(filename))

Getting an access token. This token is valid for one hour only.
Success: Access token is 5d5cc635-b292-3e24-83e1-da1a4f09b1b0
Copernicus_General_License Terms and Conditions already accepted
Query successfully submitted. Job ID is 0db96b56-2c45-46ef-9be6-3bd662b9bb5b
The Job 0db96b56-2c45-46ef-9be6-3bd662b9bb5b has not completed
The Job 0db96b56-2c45-46ef-9be6-3bd662b9bb5b has completed
Total number of products/results: 1
<Response [200]>
{'Server': 'nginx/1.15.3', 'Date': 'Mon, 14 Oct 2019 13:31:43 GMT', 'Content-Type': 'application/octet-stream', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Set-Cookie': 'route=f85a5f7075970718f910a85b79e9b7eeb8be48e4; Domain=apis.wekeo.eu; Path=/; HttpOnly, f5avrbbbbbbbbbbbbbbbb=KODPIEFANCEHKADKEECOAGHGCDKJOEEGKIIEPAAPDGHNJDNKFFBPCKEGPMJFECHOKNIAOODLEIEDIFECKHPIGGOJJIIAMHPCNIDAJOINNENHHKKJEHLKGCMPBAPGCHLL; HttpOnly; secure', 'Accept-Ranges': 'bytes', 'Access-Control-Allow-Origin': '*', 'Content-Range': 'bytes 0-422121423/422121424', '

Query successfully submitted. Job ID is f564e620-daf8-4f1c-8bd5-81f8979d7d4a
The Job f564e620-daf8-4f1c-8bd5-81f8979d7d4a has not completed
The Job f564e620-daf8-4f1c-8bd5-81f8979d7d4a has completed
Total number of products/results: 1
<Response [200]>
{'Server': 'nginx/1.15.3', 'Date': 'Mon, 14 Oct 2019 13:33:02 GMT', 'Content-Type': 'application/octet-stream', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Set-Cookie': 'route=f85a5f7075970718f910a85b79e9b7eeb8be48e4; Domain=apis.wekeo.eu; Path=/; HttpOnly, f5avrbbbbbbbbbbbbbbbb=FHCNNHJHNPICHHCIGDGFIMNPCIMPONDJGEHMJMODFEJIADCCMLGLEGJJNFIJKIEMALHDPBIMEAEDIAPOJIHONFNANKKAHDFDDIODICGFMLANOCLJMHFHAOGFOEKIMOJJ; HttpOnly; secure', 'Accept-Ranges': 'bytes', 'Access-Control-Allow-Origin': '*', 'Content-Range': 'bytes 0-51233642/51233643', 'ETag': '"52D4708B61EE0FCC676D756C50A4EBF4"', 'Access-Control-Allow-Methods': 'GET', 'Content-Disposition': 'inline;filename="S3B_SR_2_WAT____20190902T145920_20190902T154559_20190928T070007_2799